In [29]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['의도 식별', '대화 요약']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess
regex_pattern = re.compile(r'\{[^}]*\}|\(\([^)]*\)\)|\([^)]*\)')
def preprocess_text(text):
    text = regex_pattern.sub('', text)
    text = text.replace('  ', ' ').strip()
    text = text.replace(')', '').replace('(', '')
    text = text.replace('}', '').replace('{', '')
    return text
#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            
            #### data preprocess end 

            #### 의도 식별
            data = {'input': {}, 'output': {}}
            ## preprocess data from line
            utterances = []
            utterance = {
                'id': line['utterance'][0]['id'],
                'speaker_id': line['utterance'][0]['speaker_id'],
                'original_form': preprocess_text(line['utterance'][0]['original_form']),
                'intent': line['utterance'][0]['intent'] if line['utterance'][0]['intent'] else [],
            }
            summaries = {}
            for utt in line['utterance'][1:]:
                original_form = preprocess_text(utt['original_form'])
                if original_form:
                    if utt['speaker_id'] == utterance['speaker_id']:
                        utterance['original_form'] += ' ' + original_form
                        utterance['intent'] += utt['intent'] if utt['intent'] else []
                    else:
                        utterances.append(utterance)
                        utterance = {}
                        utterance['id'] = utt['id']
                        utterance['speaker_id'] = utt['speaker_id']
                        utterance['original_form'] = preprocess_text(utt['original_form'])
                        utterance['intent'] = utt['intent'] if utt['intent'] else []
                        if utt['summary']:
                            summaries[utt['id']] = utt['summary']
                
            intents = [sorted(set(utt.pop('intent'))) for utt in utterances]
            data['input']['utterances'] = utterances
            data['output']['intents'] = intents
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 의도 식별 end

            if summaries:
                #### 대화 요약
                data = {'input': {}, 'output': {}}
                ## preprocess data from line
                dialogs = []
                for utt in utterances:
                    dialogs.append(utt['original_form'])
                    if utt['id'] in summaries:
                        data['input']['utterances'] = dialogs
                        data['output']['summary'] = summaries[utt['id']]
                        dialogs = []
                        ## preprocess data from line end
                        task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
                        # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 대화 요약 end

            
            # break
        # break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 8/8 [00:02<00:00,  2.85it/s]
